# 🔐 Image Steganography using LSB + XOR Encryption
This notebook demonstrates how to hide and retrieve secret messages in images using **Least Significant Bit (LSB)** steganography combined with **XOR-based encryption**.

In [ ]:
import cv2
import numpy as np

# ===== Helper Functions =====
def xor_encrypt(message, key):
    key = [ord(k) for k in key]
    return [ord(m) ^ key[i % len(key)] for i, m in enumerate(message + '~')]  # '~' as EOF marker

def xor_decrypt(image_path, key):
    img = cv2.imread(image_path)
    decrypt = ""
    n = m = z = kl = 0
    key = [ord(k) for k in key]

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for c in range(3):
                val = img[i, j][c]
                xor_val = val ^ key[kl]
                char = chr(xor_val)
                print(f"Decrypting pixel ({n},{m},{z}): {val} XOR {key[kl]} = {xor_val} -> '{char}'")
                if char == '~':
                    print("\nDecrypted message:", decrypt)
                    return decrypt
                decrypt += char
                n += 1
                m = (m + 1) % 3
                z = (z + 1) % 3
                kl = (kl + 1) % len(key)
    print("\nDecrypted message:", decrypt)
    return decrypt

In [ ]:
def embed_encrypted_data(image_path, output_path, message, key):
    encrypted = xor_encrypt(message, key)
    img = cv2.imread(image_path)
    flat_img = img.flatten()
    bin_data = ''.join(format(byte, '08b') for byte in encrypted)
    
    if len(bin_data) > len(flat_img):
        raise ValueError("Message too large to hide in this image.")

    for i in range(len(bin_data)):
        flat_img[i] = (flat_img[i] & ~1) | int(bin_data[i])

    stego_img = flat_img.reshape(img.shape)
    cv2.imwrite(output_path, stego_img)
    print(f"Message embedded in: {output_path}")

In [ ]:
# === Example Usage ===
# 1. To embed:
# embed_encrypted_data('input.png', 'stego.png', 'secret', 'key123')

# 2. To decode:
# xor_decrypt('stego.png', 'key123')